In [1]:
import attk
import os
import numpy as np
import librosa
from IPython.display import display, Audio
import timeit
import random
from itertools import groupby
from operator import itemgetter
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
import subprocess
import unicodecsv
import urllib2

os.chdir('/home/sharedfolder/GitHub/sida/')

!mkdir /home/sharedfolder/GitHub/sida/___training_audio

os.chdir('/home/sharedfolder/GitHub/sida/___training_audio')


mkdir: cannot create directory '/home/sharedfolder/GitHub/sida/___training_audio': File exists


In [2]:
## Loading label data CSV as a list of lists

csv_url = "https://raw.githubusercontent.com/hipstas/aapb-labels/master/Universal_Background_Model_AAPB_4K_Corpus/aapb_ubm_4K_2017.csv"

csv_string = urllib2.urlopen(csv_url)

train_table = []

csv_reader = unicodecsv.reader(csv_string)

for row in csv_reader:
        train_table.append(row)

train_table[:10]+['...']

[[u'Media file basename', u'Start time', u'Duration', u'Label', u'Labeled by'],
 [u'cpb-aacip-55-19s1sc16.h264__2145.0_2155.0',
  u'2',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-29-6663z15p.h264__121.0_131.0',
  u'3',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-235-12z35scw.h264__212.0_222.0',
  u'3',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-500-7s7hv789__149.0_159.0',
  u'2',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-41-72p5hz84.h264__733.0_743.0',
  u'2',
  u'1',
  u'Noise',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-4j09w0904c__200202A21A_.h264__27.0_37.0',
  u'4',
  u'1',
  u'Male with crowd noise',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-15-8p5v698f3z__21475-2009_04_02_.h264__577.0_587.0',
  u'4',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-43-x921c1v32g__10376128___105.0_115.0',
  u'1',
  u'1',
  u'Male',
  u'Stephen R. McLaughlin'],
 [u'cpb-aacip-225-66vx0s8z.h264__1006.0_1016.

In [3]:
## Removing header row if present

if 'Media file basename' in train_table[0]:
    train_table = train_table[1:]

random.shuffle(train_table)

In [4]:
%%capture
## Excerpting labeled WAV clips

training_audio_pathname = "/Volumes/U/audio_labeler/media/ubm_clips_final_July_2017/"
out_dir = '_classes_' + training_audio_pathname.strip('/').split('/')[-1]


for row in train_table:
    try:
        basename , start, duration, class_name, labeled_by = row  ## Assigning values in row to variables
        filename = str(basename + '.mp3')
        start = float(start)
        end = float(start) + float(duration)
        out_pathname = str(os.path.join(out_dir, class_name.replace(' ','_')))
        try: 
            subprocess.call(['mkdir', '-p', out_pathname])
        except:
            pass
        attk.subclip(os.path.join(training_audio_pathname, filename), float(start), end, out_pathname) ## <- attk
    except Exception as e: 
        print(row)
        print(e)



In [5]:
"/Volumes/U/audio_labeler/media/ubm_clips_final_July_2017"

'/Volumes/U/audio_labeler/media/ubm_clips_final_July_2017'

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.
